In [22]:
import librosa
import numpy as np
import os
import skimage, cv2
import matplotlib.pyplot as plt
from scipy.signal import butter, sosfilt
import skimage.filters
import soundfile as sf
from skimage.filters import frangi
import pandas as pd
from scipy.ndimage import label

In [42]:
def count_calls(spec_path, wave_path):
    spec_img = cv2.imread(spec_path, cv2.IMREAD_GRAYSCALE)
    if spec_img is None:
        raise FileNotFoundError(f"Could not load spectrogram image at {spec_path}")
    img_norm = spec_img / 255.0

    # Horizontal edge detection (Sobel x)
    edges = cv2.Sobel(img_norm, cv2.CV_64F, 1, 0, ksize=3)
    edges = np.abs(edges)
    edges = (edges > np.percentile(edges, 99)).astype(np.uint8)

    # Morphological closing (connect broken ridges)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (140, 120))
    morphed = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel)

    structure = np.ones((3, 3), dtype=int)
    _, ridge_calls = label(morphed, structure=structure)

    # --- Audio-based short-time energy detection ---
    y, sr = librosa.load(wave_path, sr=None)

    hop_length = 512
    frame_length = 1024
    energy = np.array([
        np.sum(np.abs(y[j:j+frame_length]**2))
        for j in range(0, len(y), hop_length)
    ])
    threshold = 0.02 * np.max(energy)
    above_thresh = np.where(energy > threshold)[0]

    energy_calls = 0
    if len(above_thresh) > 0:
        energy_calls = 1
        for k in range(1, len(above_thresh)):
            if above_thresh[k] - above_thresh[k-1] > 5:
                energy_calls += 1

    # --- Combine counts ---
    final_count = (ridge_calls + energy_calls) // 2

    return final_count

In [34]:
df = pd.DataFrame(columns=["chunk_spectrogram", "call_counts"])

In [43]:
spectrograms = "mel_spectrograms/"
i = 0
for species_dir in os.listdir(spectrograms):  
    species_path = os.path.join(spectrograms, species_dir)
    
    if not os.path.isdir(species_path):  # skip if not a folder
        continue
    
    for file in os.listdir(species_path):
        if file.endswith(".png"):
            base = os.path.splitext(file)[0]
            spec_path = os.path.join(species_path, file)
            wav_path = os.path.join(species_path, base + ".wav")

            if not os.path.exists(wav_path):
                print(f"No wav found for {file}, skipping...")
                continue

            try:
                calls = count_calls(spec_path, wav_path)
                df.loc[i] = [f'{base}.png', calls]
                i += 1
            except Exception as e:
                print(f"Error with {spec_path}: {e}")
                

In [26]:
df.shape

(30015, 2)

In [44]:
df.to_csv("call_counts.csv", index=False)

In [36]:
df['call_counts'].value_counts()

call_counts
2    18538
1    11388
3       50
4       31
5        8
Name: count, dtype: int64

In [21]:
df[call_counts].value_counts()

NameError: name 'call_counts' is not defined